## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps
import json

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_DataBase/WeatherPy_Database.csv")
city_data_df.head(11)

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Punta Arenas,CL,-53.1500,-70.9167,46.40,61,40,19.57,scattered clouds
1,1,Thompson,CA,55.7435,-97.8558,42.80,31,75,6.91,broken clouds
2,2,Hilo,US,19.7297,-155.0900,78.80,61,90,3.44,overcast clouds
3,3,Hithadhoo,MV,-0.6000,73.0833,83.12,75,72,17.49,broken clouds
4,4,Grindavik,IS,63.8424,-22.4338,17.60,57,75,13.80,broken clouds
5,5,Egvekinot,RU,66.3167,-179.1667,-2.92,75,34,2.77,scattered clouds
6,6,Albany,US,42.6001,-73.9662,59.00,44,43,1.99,scattered clouds
7,7,Airai,TL,-8.9266,125.4092,60.08,95,100,3.36,overcast clouds
8,8,Tonekabon,IR,36.8163,50.8738,51.80,76,0,2.30,clear sky
9,9,Tuskegee,US,32.4240,-85.6916,73.00,28,1,8.05,clear sky


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("Whats the lowest temperature for your trip? "))
max_temp = float(input("Whats the highest temperature for your trip? "))

Whats the lowest temperature for your trip? 66
Whats the highest temperature for your trip? 95


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Hilo,US,19.7297,-155.0900,78.80,61,90,3.44,overcast clouds
3,3,Hithadhoo,MV,-0.6000,73.0833,83.12,75,72,17.49,broken clouds
9,9,Tuskegee,US,32.4240,-85.6916,73.00,28,1,8.05,clear sky
10,10,Butaritari,KI,3.0707,172.7902,82.22,79,80,16.33,broken clouds
12,12,Maceio,BR,-9.6658,-35.7353,77.00,94,20,2.30,few clouds
14,14,Waddan,LY,29.1614,16.1390,75.92,10,93,12.24,overcast clouds
15,15,Thap Khlo,TH,16.1600,100.5966,75.45,88,100,3.11,overcast clouds
18,18,Haimen,CN,23.1935,116.6142,66.20,88,90,6.71,mist
20,20,Vaini,TO,-21.2000,-175.2000,78.80,89,20,10.36,few clouds
21,21,Miri,MY,4.4148,114.0089,75.20,94,40,3.44,scattered clouds


In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()

In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Hilo,US,78.80,overcast clouds,19.7297,-155.0900,
3,Hithadhoo,MV,83.12,broken clouds,-0.6000,73.0833,
9,Tuskegee,US,73.00,clear sky,32.4240,-85.6916,
10,Butaritari,KI,82.22,broken clouds,3.0707,172.7902,
12,Maceio,BR,77.00,few clouds,-9.6658,-35.7353,
...,...,...,...,...,...,...,...
703,Cumaral,CO,80.80,broken clouds,4.2708,-73.4867,
704,Muli,MV,83.55,scattered clouds,2.9167,73.5667,
707,Orocue,CO,85.24,heavy intensity rain,4.7912,-71.3354,
708,Rio Verde De Mato Grosso,BR,83.52,broken clouds,-18.9181,-54.8442,


In [12]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for i, row in hotel_df.iterrows():
    location = f"{row['Lat']},{row['Lng']}"
    
    params = {
    "location": location,     
    "radius": 5000,
    "type": "lodging",
    "key": g_key}
    
    response = requests.get(base_url, params=params).json() 
    
    results = response['results']

    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
hotel_df.loc[i, "Hotel Name"] = results[0]["name"]
#    try:
 #        hotel_df.loc[i, "Hotel Name"] = results[0]["name"]
  #  except (IndexError):
   #     print("Hotel not found... skipping.")  

In [1]:
hotel_df.head(30)

NameError: name 'hotel_df' is not defined

In [14]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [15]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")
hotel_df.head(20)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Hilo,US,78.80,overcast clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
3,Hithadhoo,MV,83.12,broken clouds,-0.6000,73.0833,Scoop Guest House
9,Tuskegee,US,73.00,clear sky,32.4240,-85.6916,Kellogg Hotel & Conference Center
10,Butaritari,KI,82.22,broken clouds,3.0707,172.7902,Isles Sunset Lodge
12,Maceio,BR,77.00,few clouds,-9.6658,-35.7353,Poutur Pousada
14,Waddan,LY,75.92,overcast clouds,29.1614,16.1390,محل الشريف محمد العايب
15,Thap Khlo,TH,75.45,overcast clouds,16.1600,100.5966,Khao Sai Country Club Golf Course
18,Haimen,CN,66.20,mist,23.1935,116.6142,Jinghao Business Hotel
20,Vaini,TO,78.80,few clouds,-21.2000,-175.2000,Keleti Beach Resort
21,Miri,MY,75.20,scattered clouds,4.4148,114.0089,"Mega Hotel, Miri"


In [22]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [23]:
# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations_df = hotel_df[["Lat", "Lng"]]

In [24]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
#locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
weather = hotel_df["Current Description"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))